# Prophet final simulation

In [45]:
CHANGE = "constant"

In [ ]:
import pandas as pd
from prophet import Prophet
from prophet.serialize import model_from_json
import json
import plotly.graph_objs as go
import plotly.io as pio

# Load your saved Prophet model from JSON
with open('/Users/skyfano/Documents/Masterarbeit/Prediction_of_energy_prices/predictions/Prophet/prophet_model.json', 'r') as fin:
    model = model_from_json(fin.read())

# Load the data with covariates
df = pd.read_csv(f'../final_df_{CHANGE}.csv')

# Ensure the 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Filter the data to include one year before the forecast start date
forecast_start_date = '2024-06-28'
one_year_earlier = pd.to_datetime(forecast_start_date) - pd.DateOffset(years=2)
historical_data = df[(df['Date'] >= one_year_earlier) &
                     (df['Date'] <= forecast_start_date)]

# Filter the data to only include rows after June 2024 for the actual forecasting
future_data = df[df['Date'] > '2024-07-28']

# Ensure the 'ds' and 'y' columns are set up for Prophet
future_data = future_data.rename(
    columns={'Date': 'ds', 'Day_ahead_price (€/MWh)': 'y'})

# Use future_data without the target 'y' for forecasting
future_covariates = future_data.drop(columns=['y'])

# Make future predictions using the loaded model
forecast = model.predict(future_covariates)

# Plot the forecasted results
# Historical data for the past year
trace_historical = go.Scatter(
    x=df['Date'],
    y=df['Day_ahead_price (€/MWh)'],
    mode='lines',
    name='Historical',
    line=dict(color='black')
)

# Forecasted results
trace_predicted = go.Scatter(
    x=forecast['ds'],
    y=forecast['yhat'],
    mode='lines',
    name='Predicted',
    line=dict(color='blue')
)

trace_upper = go.Scatter(
    x=forecast['ds'],
    y=forecast['yhat_upper'],
    mode='lines',
    name='Upper Confidence Interval',
    line=dict(color='lightblue', width=0),
    fill='tonexty',
    fillcolor='rgba(173,216,230,0.3)'
)

trace_lower = go.Scatter(
    x=forecast['ds'],
    y=forecast['yhat_lower'],
    mode='lines',
    name='Lower Confidence Interval',
    line=dict(color='lightblue', width=0),
    showlegend=False,
    fill='tonexty',
    fillcolor='rgba(173,216,230,0.3)'
)

# Create the figure and add the traces
fig = go.Figure()
fig.add_trace(trace_historical)
fig.add_trace(trace_predicted)
fig.add_trace(trace_upper)
fig.add_trace(trace_lower)

# Customize the layout
fig.update_layout(
    title=f'Prophet forecast for {CHANGE} trend of BEVs',
    xaxis_title='Date',
    yaxis_title='Day-ahead Price (€/MWh)',
    xaxis=dict(
        # Set x-axis to start at January 1, 2023
        range=['2023-01-01', forecast['ds'].max()]
    ),
    yaxis=dict(
        range=[0, 220]  # Set y-axis range with a cutoff slightly above 200
    ),
    legend=dict(x=1.05, y=1),
    margin=dict(t=60, b=20, l=20, r=20),  # Increase top margin (t) to 60
    width=1200,
    height=450
)


# Show the plot
pio.show(fig)

In [47]:
# Save the plot as .png
pio.write_image(fig, f'forecast_prophet_{CHANGE}.png')

In [48]:
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Ascension Day,Ascension Day_lower,Ascension Day_upper,BEV_vehicles,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-07-29,57.444083,51.729349,94.419675,57.444083,57.444083,0.0,0.0,0.0,-0.151352,...,-2.072336,-2.072336,-2.072336,-1.544190,-1.544190,-1.544190,0.0,0.0,0.0,73.537033
1,2024-07-30,57.445165,53.563864,95.883639,57.445165,57.445165,0.0,0.0,0.0,-0.151352,...,1.544570,1.544570,1.544570,-2.179470,-2.179470,-2.179470,0.0,0.0,0.0,73.841744
2,2024-07-31,57.446247,47.103173,89.205048,57.446247,57.446247,0.0,0.0,0.0,-0.090589,...,3.074512,3.074512,3.074512,-2.820772,-2.820772,-2.820772,0.0,0.0,0.0,67.596365
3,2024-08-01,57.447329,45.357211,88.435874,57.447329,57.447329,0.0,0.0,0.0,3.173291,...,4.486737,4.486737,4.486737,-3.417468,-3.417468,-3.417468,0.0,0.0,0.0,65.852007
4,2024-08-02,57.448411,39.211613,83.285888,57.448411,57.448411,0.0,0.0,0.0,3.173291,...,4.146121,4.146121,4.146121,-3.922441,-3.922441,-3.922441,0.0,0.0,0.0,61.891475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,2026-07-24,58.228580,61.595462,103.737737,55.669051,60.451240,0.0,0.0,0.0,2.569994,...,4.146121,4.146121,4.146121,0.236294,0.236294,0.236294,0.0,0.0,0.0,82.467294
726,2026-07-25,58.229662,51.019603,95.205266,55.666766,60.456553,0.0,0.0,0.0,2.569994,...,-2.719560,-2.719560,-2.719560,0.194274,0.194274,0.194274,0.0,0.0,0.0,73.263965
727,2026-07-26,58.230744,46.332957,89.714214,55.664480,60.461866,0.0,0.0,0.0,2.569994,...,-8.460044,-8.460044,-8.460044,0.026722,0.026722,0.026722,0.0,0.0,0.0,67.937428
728,2026-07-27,58.231826,73.760941,114.820084,55.662195,60.467178,0.0,0.0,0.0,2.569994,...,-2.072336,-2.072336,-2.072336,-0.274823,-0.274823,-0.274823,0.0,0.0,0.0,93.540734


In [49]:
results = forecast[["ds", "yhat"]]
results.to_csv(f'prophet_results_{CHANGE}.csv', index=False)

## Plotting all results together

In [ ]:
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

# Load the CSV files
df_constant = pd.read_csv('prophet_results_constant.csv')
df_decline = pd.read_csv('prophet_results_decline.csv')
df_growth = pd.read_csv('prophet_results_growth.csv')

# Merge them on the 'ds' column
merged_df = df_constant.merge(
    df_decline, on='ds', suffixes=('_constant', '_decline'))
merged_df = merged_df.merge(df_growth, on='ds')
merged_df.rename(columns={'yhat': 'yhat_growth'}, inplace=True)

# Plot the results
trace_constant = go.Scatter(
    x=merged_df['ds'],
    y=merged_df['yhat_constant'],
    mode='lines',
    name='Constant Scenario',
    line=dict(color='blue')
)

trace_decline = go.Scatter(
    x=merged_df['ds'],
    y=merged_df['yhat_decline'],
    mode='lines',
    name='Decline Scenario',
    line=dict(color='red')
)

trace_growth = go.Scatter(
    x=merged_df['ds'],
    y=merged_df['yhat_growth'],
    mode='lines',
    name='Growth Scenario',
    line=dict(color='green')
)

# Create the figure and add the traces
fig = go.Figure()
fig.add_trace(trace_constant)
fig.add_trace(trace_decline)
fig.add_trace(trace_growth)

# Customize the layout
fig.update_layout(
    title='Forecast Comparison for Different Scenarios',
    xaxis_title='Date',
    yaxis_title='Day-ahead Price (€/MWh)',
    legend=dict(x=1.05, y=1),
    margin=dict(t=60, b=20, l=20, r=20),
    width=1200,
    height=450
)

# Show the plot
pio.show(fig)

# Save the plot as .png
pio.write_image(fig, 'forecast_comparison.png')

In [53]:
merged_df

# save a csv
merged_df.to_csv('forecast_comparison.csv', index=False)

In [ ]:
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

# Load the CSV files
df_constant = pd.read_csv('prophet_results_constant.csv')
df_decline = pd.read_csv('prophet_results_decline.csv')
df_growth = pd.read_csv('prophet_results_growth.csv')

# Merge them on the 'ds' column
merged_df = df_constant.merge(
    df_decline, on='ds', suffixes=('_constant', '_decline'))
merged_df = merged_df.merge(df_growth, on='ds')
merged_df.rename(columns={'yhat': 'yhat_growth'}, inplace=True)

# Calculate differences between scenarios
merged_df['diff_constant_decline'] = abs(
    merged_df['yhat_constant'] - merged_df['yhat_decline'])
merged_df['diff_constant_growth'] = abs(
    merged_df['yhat_constant'] - merged_df['yhat_growth'])
merged_df['diff_decline_growth'] = abs(
    merged_df['yhat_decline'] - merged_df['yhat_growth'])

# Plot the original forecasts
trace_constant = go.Scatter(
    x=merged_df['ds'],
    y=merged_df['yhat_constant'],
    mode='lines',
    name='Constant Scenario',
    line=dict(color='blue')
)

trace_decline = go.Scatter(
    x=merged_df['ds'],
    y=merged_df['yhat_decline'],
    mode='lines',
    name='Decline Scenario',
    line=dict(color='red')
)

trace_growth = go.Scatter(
    x=merged_df['ds'],
    y=merged_df['yhat_growth'],
    mode='lines',
    name='Growth Scenario',
    line=dict(color='green')
)

# Plot the differences between scenarios
trace_diff_constant_decline = go.Scatter(
    x=merged_df['ds'],
    y=merged_df['diff_constant_decline'],
    mode='lines',
    name='Difference: Constant vs Decline',
    line=dict(color='purple', dash='dot')
)

trace_diff_constant_growth = go.Scatter(
    x=merged_df['ds'],
    y=merged_df['diff_constant_growth'],
    mode='lines',
    name='Difference: Constant vs Growth',
    line=dict(color='orange', dash='dot')
)

trace_diff_decline_growth = go.Scatter(
    x=merged_df['ds'],
    y=merged_df['diff_decline_growth'],
    mode='lines',
    name='Difference: Decline vs Growth',
    line=dict(color='grey', dash='dot')
)

# Create the figure and add all traces
fig = go.Figure()
fig.add_trace(trace_constant)
fig.add_trace(trace_decline)
fig.add_trace(trace_growth)
fig.add_trace(trace_diff_constant_decline)
fig.add_trace(trace_diff_constant_growth)
fig.add_trace(trace_diff_decline_growth)

# Customize the layout
fig.update_layout(
    title='Forecast Comparison with Scenario Differences',
    xaxis_title='Date',
    yaxis_title='Day-ahead Price (€/MWh)',
    yaxis2=dict(
        title='Difference in €/MWh',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=1.05, y=1),
    margin=dict(t=60, b=20, l=20, r=20),
    width=1200,
    height=600
)

# Show the plot
pio.show(fig)

In [55]:
merged_df

,ds,yhat_constant,yhat_decline,yhat_growth,diff_constant_decline,diff_constant_growth,diff_decline_growth
0,2024-07-29,73.537033,73.502311,73.623838,0.034722,0.086805,0.121527
1,2024-07-30,73.841744,73.807022,73.928550,0.034722,0.086805,0.121527
2,2024-07-31,67.596365,67.563813,67.674490,0.032552,0.078125,0.110677
3,2024-08-01,65.852007,65.151054,67.601134,0.700953,1.749127,2.450080
4,2024-08-02,61.891475,61.190522,63.640602,0.700953,1.749127,2.450080
...,...,...,...,...,...,...,...
725,2026-07-24,82.467294,80.624851,90.145225,1.842443,7.677931,9.520374
726,2026-07-25,73.263965,71.421522,80.941896,1.842443,7.677931,9.520374
727,2026-07-26,67.937428,66.094985,75.615359,1.842443,7.677931,9.520374
728,2026-07-27,93.540734,91.698291,101.218665,1.842443,7.677931,9.520374


## Computing some metrics 

In [65]:
# Compute the mean for each diff
mean_diff_constant_decline = merged_df['diff_constant_decline'].mean()
mean_diff_constant_growth = merged_df['diff_constant_growth'].mean()
mean_diff_decline_growth = merged_df['diff_decline_growth'].mean()

mean_decline = merged_df['yhat_decline'].mean()
mean_growth = merged_df['yhat_growth'].mean()
mean_constant = merged_df['yhat_constant'].mean()

print(mean_decline, mean_growth, mean_constant)

78.70756418167407 84.14354255190311 79.86108804254938


In [ ]:
print(mean_diff_constant_decline,
      mean_diff_constant_growth, mean_diff_decline_growth)

1.1535238608753058 4.282454509353716 5.435978370229022


In [60]:
# add the name of the weekday according to the date in the ds column to the merged_df
merged_df['ds'] = pd.to_datetime(merged_df['ds'])
merged_df['weekday'] = merged_df['ds'].dt.day_name()
merged_df

,ds,yhat_constant,yhat_decline,yhat_growth,diff_constant_decline,diff_constant_growth,diff_decline_growth,weekday
0,2024-07-29,73.537033,73.502311,73.623838,0.034722,0.086805,0.121527,Monday
1,2024-07-30,73.841744,73.807022,73.928550,0.034722,0.086805,0.121527,Tuesday
2,2024-07-31,67.596365,67.563813,67.674490,0.032552,0.078125,0.110677,Wednesday
3,2024-08-01,65.852007,65.151054,67.601134,0.700953,1.749127,2.450080,Thursday
4,2024-08-02,61.891475,61.190522,63.640602,0.700953,1.749127,2.450080,Friday
...,...,...,...,...,...,...,...,...
725,2026-07-24,82.467294,80.624851,90.145225,1.842443,7.677931,9.520374,Friday
726,2026-07-25,73.263965,71.421522,80.941896,1.842443,7.677931,9.520374,Saturday
727,2026-07-26,67.937428,66.094985,75.615359,1.842443,7.677931,9.520374,Sunday
728,2026-07-27,93.540734,91.698291,101.218665,1.842443,7.677931,9.520374,Monday


In [ ]:
# compute the average difference between the scenarios per weekday
weekday_diffs = merged_df.groupby('weekday').mean()

In [ ]:
# Sort the weekdays in the correct order
weekday_diffs = weekday_diffs.reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
weekday_diffs

,ds,yhat_constant,yhat_decline,yhat_growth,diff_constant_decline,diff_constant_growth,diff_decline_growth
weekday,,,,,,,
Monday,2025-07-28 00:00:00,82.561569,81.403400,86.864075,1.158169,4.302506,5.460675
Tuesday,2025-07-29 00:00:00,86.935275,85.777891,91.232573,1.157384,4.297298,5.454681
Wednesday,2025-07-26 12:00:00,86.049205,84.902979,90.306756,1.146227,4.257551,5.403778
Thursday,2025-07-27 12:00:00,82.881657,81.732301,87.147263,1.149357,4.265605,5.414962
Friday,2025-07-28 12:00:00,80.869304,79.717986,85.135473,1.151318,4.266169,5.417487
Saturday,2025-07-29 12:00:00,73.800863,72.647145,78.084122,1.153718,4.283259,5.436976
Sunday,2025-07-30 12:00:00,65.835755,64.677342,70.140214,1.158413,4.304459,5.462872
